## Produção Diário

In [1]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os

import warnings
warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Juridico Varejo\Reportes\Diários\Produção diário Varejo Massificado - 2024.xlsx'
sheet_name = 'Dados'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

def read_and_process_excel(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    rename_df = {
        'Data reporte': 'data_reporte',
        'Data evento': 'data_evento',
        'Carteira': 'carteira',
        'CPF/CNPJ': 'cpf_cnpj',
        'Nome cliente': 'nome_cliente',
        'Escritório': 'escritorio',
        'Devedor Principal': 'tipo_devedor_princiapl',
        'Dossiê Benner': 'dossie',
        'Operações': 'operacoes',
        'Tipo ação': 'tipo_acao',
        'Indicador' : 'indicador',
        'Modalidade' : 'modalidade',
        'Revisão' : 'revisao',
        'Observações' : 'observacoes',
        'Advogado ' : 'advogado',
        'Valor envolvido' : 'valor_evolvido',
        'Qtde bens retomados' : 'qtde_bens_retomados',
        'Placa bem  móvel' : 'placa_bem_movel',
        'Data cadastro evento e doc sistema Benner' : 'data_cadastro_evento_benner', 
        'Poder judiciário' : 'poder_judiciario',
        'Rede' : 'rede',
        'Honorários' : 'honorarios' 
    }
    df.rename(columns=rename_df, inplace=True)

    df = df.dropna(subset=['data_reporte'])

    date_columns = ['data_reporte', 'data_evento']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).dt.date

    df['valor_evolvido'] = pd.to_numeric(df['valor_evolvido'].str.replace('%', '').str.replace('do faturamento', '').str.strip(), errors='coerce')

    # Criando a chave única
    df['unique_key'] = df['dossie'] + df['indicador'] + df['data_reporte'].astype(str)

    return df

def get_existing_records(db_config):
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT dossie, indicador, data_reporte FROM varejo.prod_diario"
    existing_records = pd.read_sql(query, engine)
    existing_records['unique_key'] = existing_records['dossie'] + existing_records['indicador'] + existing_records['data_reporte'].astype(str)
    return existing_records

def save_df_to_csv(df, csv_file_path):
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    client = docker.from_env()
    container = client.containers.get(container_id)

    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Salva o novo dataframe como CSV
    df_new_data.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_name, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY varejo.prod_diario FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Criar a coluna unique_key para verificação
    df['unique_key'] = df['dossie'] + df['indicador'] + df['data_reporte'].astype(str)

    # Fazer a mesma combinação no banco de dados para comparar
    existing_records['unique_key'] = existing_records['dossie'] + existing_records['indicador'] + existing_records['data_reporte'].astype(str)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['unique_key'].isin(existing_records['unique_key'])]

    if not df_new_data.empty:
        # Remover a coluna unique_key antes de carregar os dados
        df_new_data.drop(columns=['unique_key'], inplace=True)

        # Se houver novos dados, insira-os no banco
        insert_new_data(df_new_data, db_config, container_id, 'dados.csv')
        os.remove('dados.csv')
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")


COPY 2

Novos dados carregados com sucesso!


## Levantamento Alvará Julho 2024

In [2]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os
import re

import warnings
warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Juridico Varejo\Reportes\Alvarás_Sisbajud positivo\Cópia de Levantamento Alvará Julho 2024.xlsx'
sheet_name = 'Planilha1'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

# Dicionário para renomear colunas
rename_dict = {
    'REPORTE ': 'reporte',
    'REDE': 'rede',
    'CARTEIRAS': 'carteira',
    'SEGMENTO ': 'segmento',
    'GESTOR': 'gestor',
    'ESCRITÓRIO': 'escritorio',
    'CLIENTE ': 'cliente',
    'CNPJCPF': 'cnpjcpf',
    'OP19': 'op19',
    ' GOVERNAMENTAL ': 'governamental',
    'DOSSIÊ': 'dossie',
    'AÇÃO': 'acao',
    'DATA DO PEDIDO DA PENHORA': 'data_do_pedido_da_penhora',
    'DATA DO DEFERIMENTO DO PEDIDO': 'data_do_deferimento_do_pedido',
    'DATA DO BLOQUEIO': 'data_do_bloqueio',
    'VALOR BLOQUEADO': 'valor_bloqueado',
    'DEVEDOR INTIMADO ': 'devedor_estimado',
    'IMPUGNAÇÃO APRESENTADA ': 'impugnacao_apresentada',
    'IMPUGNAÇÃO ACOLHIDA ': 'impugnacao_acolhida', 
    'QUAL MATÉRIA DA IMPUGNAÇÃO?': 'qual_materia_da_impugnacao',
    ' DEFERIMENTO DE LEVANTAMENTO': 'deferimento_de_levantamento',
    'RECURSO INTERPOSTO? ': 'recurso_interposto' ,
    'EXPECTATIVA DE LEVANTAMENTO' : 'expectativa_de_levantamento',
    'STATUS ' : 'status',
    'VALOR LEVANTADO' : 'valor_levantado',
    'OBSERVAÇÃO ' : 'observacao'
}

def read_and_process_excel(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Renomeia as colunas de acordo com o dicionário
    df.rename(columns=rename_dict, inplace=True)

    # Imprime os nomes das colunas após a renomeação para verificação
    print("Nomes das colunas após renomeação:")
    print(df.columns)

    # Verificar se a coluna 'data_do_pedido_da_penhora' e outras estão presentes
    required_columns = ['data_do_pedido_da_penhora', 'reporte', 'data_do_deferimento_do_pedido', 'data_do_bloqueio']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise KeyError(f"As seguintes colunas estão faltando: {', '.join(missing_columns)}")

    # Drop rows where 'data_do_pedido_da_penhora' is NaN
    df = df.dropna(subset=['data_do_pedido_da_penhora'])

    # Converter colunas de data
    for col in required_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

    # Converter colunas para strings e criar a chave única
    df['dossie'] = df['dossie'].astype(str)
    df['cnpjcpf'] = df['cnpjcpf'].astype(str)
    df['reporte'] = df['reporte'].astype(str)
    df['unique_key'] = df['dossie'] + df['cnpjcpf'] + df['reporte']

    return df


def get_existing_records(db_config):
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT dossie, cnpjcpf, reporte FROM varejo.levantamento_alvara"
    existing_records = pd.read_sql(query, engine)
    existing_records['unique_key'] = existing_records['dossie'] + existing_records['cnpjcpf'] + existing_records['reporte'].astype(str)
    return existing_records

def save_df_to_csv(df, csv_file_path):
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    client = docker.from_env()
    container = client.containers.get(container_id)

    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados2.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Salva o novo dataframe como CSV
    df_new_data.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_name, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY varejo.levantamento_alvara FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Criar a coluna unique_key para verificação
    df['unique_key'] = df['dossie'] + df['cnpjcpf'] + df['reporte'].astype(str)

    # Fazer a mesma combinação no banco de dados para comparar
    existing_records['unique_key'] = existing_records['dossie'] + existing_records['cnpjcpf'] + existing_records['reporte'].astype(str)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['unique_key'].isin(existing_records['unique_key'])]

    if not df_new_data.empty:
        # Remover a coluna unique_key antes de carregar os dados
        df_new_data.drop(columns=['unique_key'], inplace=True)

        # Se houver novos dados, insira-os no banco
        insert_new_data(df_new_data, db_config, container_id, 'dados2.csv')
        os.remove('dados2.csv')
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")


Nomes das colunas após renomeação:
Index(['reporte', 'rede', 'carteira', 'segmento', 'gestor', 'escritorio',
       'cliente', 'cnpjcpf', 'op19', 'governamental', 'dossie', 'acao',
       'data_do_pedido_da_penhora', 'data_do_deferimento_do_pedido',
       'data_do_bloqueio', 'valor_bloqueado', 'devedor_estimado',
       'impugnacao_apresentada', 'impugnacao_acolhida',
       'qual_materia_da_impugnacao', 'deferimento_de_levantamento',
       'recurso_interposto', 'expectativa_de_levantamento', 'status',
       'valor_levantado', 'observacao'],
      dtype='object')
Nenhum novo dado para carregar.


## BBC_GARANTIA_IMOV 12072018

In [3]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os
import re

import warnings
warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Juridico Varejo\Reportes\Imóveis\Cópia de Cópia de BBC_GARANTIA_IMOV 12072018.xlsx'
sheet_name = 'Planilha1'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

# Dicionário para renomear colunas
rename_dict = {
    'OP19': 'op_19',
    'Dossie_Ativa_1': 'dossie_ativa_1',
    'Dossie_Ativa_2': 'dossie_ativa_2',
    'Dossie_Ativa_3': 'dossie_ativa_3',
    'Dossie_Ativa_4': 'dossie_ativa_4',
    'Dossie_Ativa_5': 'dossie_ativa_5',
    'NumeroOperacaoOrigem ': 'operacao_original',
    'NumeroOperacao': 'operacao',
    'Cliente': 'cliente',
    'cd_cpfcnpj ': 'cpf_cnpj',
    'Tipo_Pessoa': 'tipo_pessoa',
    'Produto': 'produto',
    'Escritorio_Ranking': 'escritorio',
    'Tipo_Carteira': 'carteira',
    'SALDO_ATRASO_OPER': 'saldo_atraso_ope',
    'DESC_CATEGORIA_GARANTIA': 'desc_categoria_garantia',
    'DESC_TIPO_GARANTIA': 'desc_tipo_garantia',
    'STATUS ATUAL PROCEDIMENTO EXTRAJUDICIAL DE CONSOLIDAÇÃO - ALIENAÇÃO FIDUCIÁRIA': 'status_consolidacao_af',
    'STATUS ATUAL PROCESSO DE EXECUÇÃO': 'status_processo_execucao'
}

def read_and_process_excel(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Renomeia as colunas de acordo com o dicionário
    df.rename(columns=rename_dict, inplace=True)

    # Imprime os nomes das colunas após a renomeação para verificação
    print("Nomes das colunas após renomeação:")
    print(df.columns)
              
    # Remover colunas indesejadas
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Converter 'saldo_atraso_ope' para numérico, forçando erros a se tornarem NaN
    df['saldo_atraso_ope'] = pd.to_numeric(df['saldo_atraso_ope'], errors='coerce')

    # Remover ou tratar valores NaN
    df = df.dropna(subset=['saldo_atraso_ope'])

    # Dropar colunas completamente vazias
    df = df.dropna(axis=1, how='all')
    
    # Converter colunas para strings e criar a chave única
    df['op_19'] = df['op_19'].astype(str)
    df['dossie_ativa_1'] = df['dossie_ativa_1'].astype(str)
    df['saldo_atraso_ope'] = df['saldo_atraso_ope'].astype(str)
    df['unique_key'] = df['op_19'] + df['dossie_ativa_1'] + df['saldo_atraso_ope']

    return df

def get_existing_records(db_config):
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT op_19, dossie_ativa_1, saldo_atraso_ope FROM varejo.bbc_garantia_imov"
    existing_records = pd.read_sql(query, engine)
    existing_records['unique_key'] = existing_records['op_19'] + existing_records['dossie_ativa_1'] + existing_records['saldo_atraso_ope'].astype(str)
    return existing_records

def save_df_to_csv(df, csv_file_path):
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    client = docker.from_env()
    container = client.containers.get(container_id)

    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados3.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Salva o novo dataframe como CSV
    df_new_data.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_name, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY varejo.bbc_garantia_imov FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Criar a coluna unique_key para verificação
    df['unique_key'] = df['op_19'] + df['dossie_ativa_1'] + df['saldo_atraso_ope'].astype(str)

    # Fazer a mesma combinação no banco de dados para comparar
    existing_records['unique_key'] = existing_records['op_19'] + existing_records['dossie_ativa_1'] + existing_records['saldo_atraso_ope'].astype(str)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['unique_key'].isin(existing_records['unique_key'])]

    if not df_new_data.empty:
        # Remover a coluna unique_key antes de carregar os dados
        df_new_data.drop(columns=['unique_key'], inplace=True)

        # Se houver novos dados, insira-os no banco
        insert_new_data(df_new_data, db_config, container_id, 'dados3.csv')
        os.remove('dados3.csv')
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")


Nomes das colunas após renomeação:
Index(['op_19', 'dossie_ativa_1', 'dossie_ativa_2', 'dossie_ativa_3',
       'dossie_ativa_4', 'dossie_ativa_5', 'NumeroOperacaoOrigem', 'operacao',
       'cliente', 'cd_cpfcnpj', 'tipo_pessoa', 'produto', 'escritorio',
       'carteira', 'saldo_atraso_ope', 'desc_categoria_garantia',
       'desc_tipo_garantia', 'status_consolidacao_af',
       'status_processo_execucao', 'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')
Nenhum novo dado para carregar.


## Controle de Formalizações Acordos

In [4]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os
import re

import warnings
warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Juridico Varejo\Controle De Formalizações\2024\Controle formalizações 2024.xlsx'
sheet_name = 'Dados'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

# Dicionário para renomear colunas
rename_dict = {
    'Índice': 'indice',
    'Negociador': 'negociador',
    'Especificidade': 'especificidade',
    'Acordo nº': 'numero_acordo',
    'Operações': 'operacoes',
    'Cliente': 'cliente',
    'CPF/CNPJ': 'cpf_cnpj',
    'Tipo pessoa': 'tipo_pessoa',
    'Dossiê': 'dossie',
    'Honorário?': 'honorarios',
    'Reserva de honorários': 'reserva_honorarios',
    'Revisional?': 'revisional',
    'Alvará?': 'alvara',
    'Data confecção minuta': 'data_confeccao_minuta',
    'Previsão pgto entrada': 'previsao_pagamento_entrada',
    'Status': 'status',
    'Data implantação': 'data_implantacao',
    'Data indexeção Benner': 'data_indexacao_benner',
    'Comunicar Acordo': 'comunicar_acordo',
    'Data homologação' : 'data_homologação',
    'Obs' : 'obs'
}

def read_and_process_excel(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Renomeia as colunas de acordo com o dicionário
    df.rename(columns=rename_dict, inplace=True)

    # Imprime os nomes das colunas após a renomeação para verificação
    print("Nomes das colunas após renomeação:")
    print(df.columns)

    date_columns = ['data_confeccao_minuta', 'previsao_pagamento_entrada', 'data_implantacao', 'data_indexacao_benner', 'comunicar_acordo', 'data_homologação']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

    # Remover colunas indesejadas
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Remover ou tratar valores NaN
    df = df.dropna(subset=['indice'])

    # Dropar colunas completamente vazias
    df = df.dropna(axis=1, how='all')
    
    # Converter colunas para strings e criar a chave única
    df['indice'] = df['indice'].astype(str)
    df['dossie'] = df['dossie'].astype(str)
    df['status'] = df['numero_acordo'].astype(str)
    df['unique_key'] = df['indice'] + df['dossie'] + df['numero_acordo']

    return df

def get_existing_records(db_config):
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT indice, dossie, numero_acordo FROM varejo.controle_formalizacoes_acordos"
    existing_records = pd.read_sql(query, engine)
    existing_records['unique_key'] = existing_records['indice'] + existing_records['dossie'] + existing_records['numero_acordo'].astype(str)
    return existing_records

def save_df_to_csv(df, csv_file_path):
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    client = docker.from_env()
    container = client.containers.get(container_id)

    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados4.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Salva o novo dataframe como CSV
    df_new_data.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_name, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY varejo.controle_formalizacoes_acordos FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Criar a coluna unique_key para verificação
    df['unique_key'] = df['indice'] + df['dossie'] + df['numero_acordo'].astype(str)

    # Fazer a mesma combinação no banco de dados para comparar
    existing_records['unique_key'] = existing_records['indice'] + existing_records['dossie'] + existing_records['numero_acordo'].astype(str)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['unique_key'].isin(existing_records['unique_key'])]

    if not df_new_data.empty:
        # Remover a coluna unique_key antes de carregar os dados
        df_new_data.drop(columns=['unique_key'], inplace=True)

        # Se houver novos dados, insira-os no banco
        insert_new_data(df_new_data, db_config, container_id, 'dados4.csv')
        os.remove('dados4.csv')
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")


Nomes das colunas após renomeação:
Index(['indice', 'negociador', 'especificidade', 'numero_acordo', 'operacoes',
       'cliente', 'cpf_cnpj', 'tipo_pessoa', 'dossie', 'honorarios',
       'reserva_honorarios', 'revisional', 'alvara', 'data_confeccao_minuta',
       'previsao_pagamento_entrada', 'status', 'data_implantacao',
       'data_indexacao_benner', 'comunicar_acordo', 'data_homologação', 'obs'],
      dtype='object')
COPY 10

Novos dados carregados com sucesso!


## Ajuizamentos 2024

In [5]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os
import re

import warnings
warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Juridico Varejo\Ajuizamento\Produção\Produção 2024\Planilha produção Varejo - Ajuizamentos 2024.xlsx'
sheet_name = 'Dados'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

# Dicionário para renomear colunas
rename_dict = {
    'Credenciado': 'credenciado',
    'Data do Ajuizamento': 'data_ajuizamento',
    'Cliente': 'cliente',
    'CPF/CNPJ': 'cpf_cnpj',
    'Cliente': 'cliente',
    'Tipo pessoa': 'tipo_pessoa',
    'UF': 'uf',
    'Operação': 'operacao',
    'Dias de atraso ': 'dias_atraso',
    'Dossiê': 'dossie',
    'Tipo de Ação': 'tipo_acao',
    'Carteira': 'carteira',
    'Segmento': 'segmento',
    'Garantia?': 'garantia',
    'Pesquisa Patrimonial?': 'pesquisa_patrimonial',
    'Sistema': 'sistema',
    'Valor da Ação': 'valor_acao',
    'Rede': 'rede'
}

def read_and_process_excel(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Renomeia as colunas de acordo com o dicionário
    df.rename(columns=rename_dict, inplace=True)

    # Imprime os nomes das colunas após a renomeação para verificação
    print("Nomes das colunas após renomeação:")
    print(df.columns)

    date_columns = ['data_ajuizamento']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

    # Remover colunas indesejadas
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Remover ou tratar valores NaN
    df = df.dropna(subset=['data_ajuizamento'])

    # Dropar colunas completamente vazias
    df = df.dropna(axis=1, how='all')
    
    # Converter colunas para strings e criar a chave única
    df['data_ajuizamento'] = df['data_ajuizamento'].astype(str)  # Converter para string
    df['dossie'] = df['dossie'].astype(str)
    df['cpf_cnpj'] = df['cpf_cnpj'].astype(str)
    df['unique_key'] = df['data_ajuizamento'] + df['dossie'] + df['cpf_cnpj']

    return df

def get_existing_records(db_config):
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT data_ajuizamento, dossie, cpf_cnpj FROM varejo.ajuizamentos_2024"
    existing_records = pd.read_sql(query, engine)
    
    # Converter a coluna de data para string antes de concatenar
    existing_records['data_ajuizamento'] = existing_records['data_ajuizamento'].astype(str)
    existing_records['cpf_cnpj'] = existing_records['cpf_cnpj'].astype(str)
    existing_records['dossie'] = existing_records['dossie'].astype(str)
    
    existing_records['unique_key'] = existing_records['data_ajuizamento'] + existing_records['dossie'] + existing_records['cpf_cnpj']
    return existing_records

def save_df_to_csv(df, csv_file_path):
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    client = docker.from_env()
    container = client.containers.get(container_id)

    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados5.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Salva o novo dataframe como CSV
    df_new_data.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_name, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY varejo.ajuizamentos_2024 FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Criar a coluna unique_key para verificação no DataFrame processado
    df['unique_key'] = df['data_ajuizamento'] + df['dossie'] + df['cpf_cnpj'].astype(str)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['unique_key'].isin(existing_records['unique_key'])]

    if not df_new_data.empty:
        # Remover a coluna unique_key antes de carregar os dados
        df_new_data.drop(columns=['unique_key'], inplace=True)

        # Se houver novos dados, insira-os no banco
        insert_new_data(df_new_data, db_config, container_id, 'dados5.csv')
        os.remove('dados5.csv')
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")

Nomes das colunas após renomeação:
Index(['Índice', 'credenciado', 'data_ajuizamento', 'cliente', 'cpf_cnpj',
       'tipo_pessoa', 'uf', 'operacao', 'dias_atraso', 'dossie', 'tipo_acao',
       'carteira', 'segmento', 'garantia', 'pesquisa_patrimonial', 'sistema',
       'valor_acao', 'rede'],
      dtype='object')
COPY 4

Novos dados carregados com sucesso!


## Redirecionamentos Unificado

In [6]:
import pandas as pd
import docker
import tarfile
from io import BytesIO
from sqlalchemy import create_engine
import os
import warnings

warnings.filterwarnings('ignore')

# Configurações do banco de dados
db_config = {
    'dbname': 'rama',
    'user': 'pcecere',
    'password': '244049',
    'host': 'localhost',
    'port': 5434
}

# Caminho para o arquivo Excel e CSV
excel_file_path = r'X:\Recuperação\Indicações\Indicações 2024\Redirecionamentos - 2024\Redirecionamentos Unificado.xlsx'
sheet_name = 'Planilha1'  # Altere para o nome da aba correta
container_id = '223c59c1268f'  # Substitua pelo ID do seu container

# Dicionário para renomear colunas
rename_dict = {
    'Data de indicação': 'data_indicacao',
    'Cliente': 'cliente',
    'CPF/CNPJ': 'cpf_cnpj',
    'Situação': 'situacao',
    'Operação': 'operacao',
    'UF': 'uf',
    'Carteira': 'carteira',
    'Solicitado em': 'data_solicitacao',
    'Regularizado?': 'regularizado'
}

def read_and_process_excel(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    # Renomeia as colunas de acordo com o dicionário
    df.rename(columns=rename_dict, inplace=True)

    # Imprime os nomes das colunas após a renomeação para verificação
    print("Nomes das colunas após renomeação:")
    print(df.columns)

    date_columns = ['data_indicacao', 'data_solicitacao']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

    # Remover colunas indesejadas
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Remover ou tratar valores NaN
    df = df.dropna(subset=['data_indicacao'])

    # Dropar colunas completamente vazias
    df = df.dropna(axis=1, how='all')
    
    # Converter colunas para strings e criar a chave única
    df['data_indicacao'] = df['data_indicacao'].astype(str)  # Converter para string
    df['operacao'] = df['operacao'].astype(str)
    df['cpf_cnpj'] = df['cpf_cnpj'].astype(str)
    df['unique_key'] = df['data_indicacao'] + df['operacao'] + df['cpf_cnpj']

    return df

def get_existing_records(db_config):
    engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")
    query = "SELECT data_indicacao, operacao, cpf_cnpj FROM varejo.redirecionamentos_unificado"
    existing_records = pd.read_sql(query, engine)

    # Convertendo as colunas de data para string antes da concatenação
    existing_records['data_indicacao'] = existing_records['data_indicacao'].astype(str)
    existing_records['operacao'] = existing_records['operacao'].astype(str)
    existing_records['cpf_cnpj'] = existing_records['cpf_cnpj'].astype(str)

    existing_records['unique_key'] = existing_records['data_indicacao'] + existing_records['operacao'] + existing_records['cpf_cnpj']
    return existing_records

def save_df_to_csv(df, csv_file_path):
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

def copy_csv_to_docker_container(csv_file_path, container_id):
    client = docker.from_env()
    container = client.containers.get(container_id)

    with BytesIO() as tar_stream:
        with tarfile.open(fileobj=tar_stream, mode='w') as tar:
            tar.add(csv_file_path, arcname='dados6.csv')
        tar_stream.seek(0)
        container.put_archive('/tmp', tar_stream)

def execute_command_in_container(container_id, command):
    client = docker.from_env()
    container = client.containers.get(container_id)
    exec_id = container.exec_run(command)
    print(exec_id.output.decode())

def insert_new_data(df_new_data, db_config, container_id, csv_file_name):
    # Remove a coluna 'indice' se estiver presente
    if 'indice' in df_new_data.columns:
        df_new_data.drop(columns=['indice'], inplace=True)
    
    # Salva o novo dataframe como CSV
    df_new_data.to_csv(csv_file_name, index=False, encoding='utf-8-sig')

    # Copia o CSV para o container Docker
    copy_csv_to_docker_container(csv_file_name, container_id)

    # Carrega os novos dados
    load_csv_command = f"psql -U {db_config['user']} -d {db_config['dbname']} -c \"COPY varejo.redirecionamentos_unificado (data_indicacao, cliente, cpf_cnpj, situacao, operacao, uf, carteira, data_solicitacao, regularizado) FROM '/tmp/{csv_file_name}' DELIMITER ',' CSV HEADER\""
    execute_command_in_container(container_id, load_csv_command)

if __name__ == "__main__":
    # Ler e processar o Excel
    df = read_and_process_excel(excel_file_path, sheet_name)

    # Obter os registros existentes no banco de dados
    existing_records = get_existing_records(db_config)

    # Criar a coluna unique_key para verificação
    df['unique_key'] = df['data_indicacao'] + df['operacao'] + df['cpf_cnpj'].astype(str)

    # Fazer a mesma combinação no banco de dados para comparar
    existing_records['unique_key'] = existing_records['data_indicacao'] + existing_records['operacao'] + existing_records['cpf_cnpj'].astype(str)

    # Comparar os dados novos com os existentes para identificar novos dossiês
    df_new_data = df[~df['unique_key'].isin(existing_records['unique_key'])]

    if not df_new_data.empty:
        # Remover a coluna unique_key antes de carregar os dados
        df_new_data.drop(columns=['unique_key'], inplace=True)

        # Se houver novos dados, insira-os no banco
        insert_new_data(df_new_data, db_config, container_id, 'dados6.csv')
        os.remove('dados6.csv')
        print("Novos dados carregados com sucesso!")
    else:
        print("Nenhum novo dado para carregar.")


Nomes das colunas após renomeação:
Index(['data_indicacao', 'cliente', 'cpf_cnpj', 'situacao', 'operacao', 'uf',
       'carteira', 'data_solicitacao', 'regularizado'],
      dtype='object')
Nenhum novo dado para carregar.
